In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from collections import Counter
import math
import numpy as np
import matplotlib.colors as mcolors
import pandas as pd
import re
from networkx import density 
from community import community_louvain
from PIL import Image

## Data creating & Data cleaning
Полученные при скачивании "графа друзей" из vk.com (я взяла только своих друзей) данные пришлось немного почистить, прежде чем работать с ними. Так, изначально в графе было 603 нода, среди которых были в том числе и удаленные пользователи. Поскольку их был небольшое число, они были удалены вручную после фильтрации в Gephi, где изначально граф был открыт. 
Из Gephi, где так же были посчитаны изначально некоторые метрики и выделены кластеры согласно показателям модулярности, были выкачаны .csv таблицы с данными по узлам и данными по ребрам. Поскольку в таблице ребер появляются только те люди, у которых есть хотя бы одна связь (замечу, себя я в граф не включала), при визуализации в networkx не были отражены друзья-одиночки (честно говоря, это задание заставило задуматься, а кто вообще эти люди). В результате, узлов всего 572.
Также стоит отметить, что в таблице ребер были указаны id узлов, у которых есть связь, а не их лейблы (имена_фамилии). Для удобства я поменяла id на labels, чтобы при визуализации в networkx мы все-таки понимали, кто есть кто.
Кроме того, мне удобнее работать сразу с csv файлами, а не открывать .gexf файл.

In [ ]:
nodes = pd.read_csv("Friends_graph_nodes.csv") 
edges = pd.read_csv("Friends_graph_edges.csv") 
clean_nodes = nodes[['Id','Label']]
clean_edges = edges[['Source','Target']]

In [ ]:
dict_id_labels = {row[1]['Id'] : row[1]['Label'] for row in clean_nodes.iterrows()}

In [ ]:
clean_edges_with_names = clean_edges.copy()
clean_edges_with_names = clean_edges.applymap(lambda id: dict_id_labels[id])

In [ ]:
print(clean_edges_with_names)

In [ ]:
graph=nx.from_pandas_edgelist(clean_edges_with_names, 'Source', 'Target')

## Basic statistics - number of nodes, edges, components, density

Здесь я предлагаю посчитать статистику для моего неориентированного графа. Ориентированным я не вижу смысла его делать, потому что мы не можем определить направление "отношений", если ты друг, то это взаимно.

In [ ]:
def visualize_graph(graph, title, color):
    nx.kamada_kawai_layout(graph) 
    pos = nx.kamada_kawai_layout(graph)
    plt.figure(figsize=(36, 30))
    plt.title(title, size = '36', weight = 'bold', color = 'black')
    nx.draw(graph, pos, with_labels=True, node_color=color, edge_color='darkgrey')

In [ ]:
def metrics(graph):
    print('Number of nodes:', graph.number_of_nodes())
    print('Number of edges:', graph.number_of_edges())
    print('Diameter of graph:', nx.diameter(graph))
    print('Number of components:', nx.number_connected_components(graph))
    print('The density of graph is:', nx.density(graph))
    print('The average degree of graph is:', len(graph.edges())/len(graph.nodes()))

In [ ]:
visualize_graph(graph, 'General graph', 'steelblue')
metrics(graph)

После визуализации графа мы в целом можем наблюдать как минимум два больших кластера, на которые он разделился - кластер школы(верхний) и кластер университета(нижний). В дальнейшем будет видно, что кластеров гораздо больше, и что даже верхнее большое облако разделилось на два сообщества. Сейчас же мы можем сказать, что у нас довольно высокий показатель связности у узлов и что у нас на графе можно наблюдать реализацию правила шести рукопожатий (если бы в графе была я, то жиаметр был бы меньше 6 + вероятно, для его уменьшения надо из графа убрать такие узлы, как, например, "Себом Ли" - т.е. узлы с одной связью). Плотность графа не очень большая в сравнении с плотностью, посчитанной ниже. 

In [ ]:
sub_graph = nx.algorithms.core.k_core(graph, k = 20)
sub_graph  = sub_graph.subgraph(max(nx.connected_components(sub_graph), key=len))
visualize_graph(sub_graph, 'Sub-graph, k = 20', 'crimson')
metrics(sub_graph)

Применив k-core decomposition к нашему общему графу, мы отфильтровали те ноды, у которых значение нагрузки меньше 20. В результате мы более отчетливо получили те два кластера, о которых мы говорили выше. Более того, наши показатели метрик нагрузки и плотности увеличились (плотность более, чем в 3 раза), что в целом не удивительно, потому что в школе и университете все общаются со всеми.

## Descriptive analysis

In [ ]:
degree_sequence = sorted([degree for node, degree in graph.degree()], reverse=True)  # degree sequence
degree_count = Counter(degree_sequence)
degree, count = zip(*degree_count.items())

plt.title("Log-log of degree desctribution")
plt.scatter([math.log(i) for i in degree_count.keys()], [math.log(i) for i in degree_count.values()])
plt.show()

In [ ]:
'''апросимирующая прямая'''
x = np.array([math.log(i) for i in degree_count.keys()])
y = np.array([math.log(i) for i in degree_count.values()])
m, b = np.polyfit(x, y, 1)
plt.scatter(x, y)
plt.plot(x, m*x + b)
plt.title("Log-log of degree desctribution with linear regression")
plt.show()

print('m равно:', m)
print('b равно:', b)

$$\log y = m \log x + b \Leftrightarrow y = e^{m \log x + b} = e^b(e^{\log x})^m = e^b x^m$$

In [ ]:
fig, ax = plt.subplots(figsize=(40,20))
plt.bar(degree, count, width=1, color="steelblue")
plt.plot(degree[:-4], (np.e**b * degree**m)[:-4], color='red', linewidth=5, label='Степенная зависимость degree от count')
plt.legend(fontsize = 40)
plt.title("Degree Histogram", fontsize=40)
plt.ylabel("Count", fontsize=40)
plt.xlabel("Degree", fontsize=40)
ax.set_xticks(degree)
ax.set_yticks(count)
plt.show()

Мы анализируем граф социальных сетей, и в данном случае наше распределение похоже на степенное (как и должно быть). Мы видим, что число вершин с данным degree убывает со скоростью $x^{-0.7}$ с ростом degree, т.е. практически по закону Ципфа ($x^{-1}$).

## Centralities

In [ ]:
'''Подсчет показателей разных центральностей и выведелние результатов в датафрейме'''
degree = nx.degree_centrality(graph)
closeness = nx.closeness_centrality(graph)
eigenvector = nx.eigenvector_centrality(graph)
betweenness = nx.betweenness_centrality(graph)
df = pd.DataFrame([degree, closeness, eigenvector, betweenness], index=['degree','closeness', 'eigenvector', 'betweenness']).T
df.sort_values(by=['betweenness'])

In [ ]:
'''Функция для визуализации графа согласно показателю какой-нибудь центральности'''
def draw(G, pos, measures, measure_name):
    
    fig = plt.figure(figsize=(50,40))
    nodes = nx.draw_networkx_nodes(G, pos, cmap=plt.cm.BrBG, node_color=list(measures.values()),
                                   nodelist=measures.keys(), node_size=[v * 3000 for v in measures.values()])
    # nodes.set_norm(mcolors.SymLogNorm(linthresh=0.01, linscale=1, base=10))
    labels = nx.draw_networkx_labels(G, pos, font_color='black', font_size='11')
    edges = nx.draw_networkx_edges(G, pos, edge_color = 'darkgrey')

    plt.title(measure_name, color='black', size = '48', weight = 'bold')
    plt.colorbar(nodes, orientation = 'horizontal')
    plt.axis('off')
    fig.set_facecolor('white')
    plt.show()

In [ ]:
pos = nx.kamada_kawai_layout(graph)
draw(graph, pos, nx.degree_centrality(graph),'Degree Centrality')
draw(graph, pos, nx.closeness_centrality(graph),'Closeness Centrality')
draw(graph, pos, nx.eigenvector_centrality(graph),'Eigenvector Centrality')
draw(graph, pos, nx.betweenness_centrality(graph),'Betweenness Centrality')

* **Degree Centrality**

Довольно непоказательная мера, которая не совсем кореллирует с betweenness centrality (хотя обычно так). Самые большие показатели - у людей, с которыми у меня наибольшее число общих знакомых. В школе я больше взаимодействовала с людьми, и потому все люди, у которых высокий показатель, относятся к кластеру школы.

* **Closeness Centrality**
Вспомним, что Closeness Centrality определяется по формуле

$$C(x)=\frac{N}{\sum_{y} d(y, x)}$$

Т.е. один делить на среднее расстояние от всех вершин. Чем меньше сумма расстояний от узла, тем выше значение центральности => тем ближе к центру графа будет находиться узел (узел с наименьшей суммой вершин будет находиться "посередине" графа). Те ноды, что окрашены в более тёмный обладают наименьшим расстоянием до любых остальных узлов (судя по нашей шкале, их центральность 1/2, т.е. расстояние или среднее равно 2). Это довольно интуитивно, потому что в кластере школы почти все связаны друг с другом, а, следовательно, до остальных меньших кластеров может быть всего одно рукопожатие. В университете же все общаются маленькими группками, и потому там показатели closeness centrality ниже.

* **Eigenvector Centrality**

Суть в том, что большой вес придается вершинам, у которых много связей с также важными вершинами. Те люди, у которых наивысшее значение Degree Centrality (т.е. люди общительные, а я могу заверить, что Иван Марков, Илья Литкенс, Лиза Подколзина, Александр Геворкянц и прочие именно такие, потому что либо это учителя, которые общаются со всеми, либо звезды школьной "эстрады"), обладают и наивысшим Eigenvector Centrality, что закономерно. В коммьюнити университетском, где люди тусуются в маленьких компаниях, кажлый человек связан с малым числом людей, а потому там важных вершин вообще нет. 

* **Betweenness Centrality**

Наконец, степень посредничества наивысшая у Анны Новиковой - моей одноклассницы и одновременно однокурсницы, которая соединяет два больших кластера. 

## Communities

Я предлагаю посмотреть на то, как выделяются сообщества согласно алгоритму Гирвана-Ньюмана и методу Лувена. 

In [ ]:
from networkx.algorithms.community.centrality import girvan_newman
communities = girvan_newman(graph)
result = {i:words for i, words in enumerate(tuple(sorted(c) for c in next(communities)))}

subset_color = [
    "teal",
    "orchid",
    "lightcolar",
    "mediumaquamarine",
]
color = [ ]
for v in graph.nodes():
    for i, words in result.items():
        if v in words:
            color.append(subset_color[i])
plt.figure(figsize=(40, 30))
nx.draw(graph, pos, node_color=color, with_labels=True, node_size=300, edge_color='darkgray', font_size='18')
plt.title('Girvan–Newman algorithm', size = '36', weight = 'bold', color = 'black')
plt.show()

In [ ]:
partition = community_louvain.best_partition(graph)
modularity = community_louvain.modularity(partition, graph)

print('Количество сообществ:', max(Counter(partition).values()))
print('Показатель моделярности:', modularity)

fig = plt.figure(figsize = (40,30))
values = [partition.get(node) for node in graph.nodes()]
nx.draw(graph, pos, cmap='Set3', node_color = values, node_size=300,
               with_labels=False, font_size = '18', font_color = 'black', edge_color = 'dimgray')
plt.title('Louvain method', size = '36', weight = 'bold', color = 'black')
fig.set_facecolor('white')
plt.show()

Честно говоря, Гирван-Ньюман дал далеко непоказательные результаты: у нас выделилось два сообщества, одно из которых включает в себя только трех людей из одной семьи с одинокавыми фамилиями. Не совсем понятно, почему выделились именно они, хотя возможно из-за числа общих у нас друзей (что мы являемся общими друзьями друг для друга). 
Метод лувена в разы показательнее. Выше я уже писала, что большое облако нодов разделится на две части. Причем эти части практически равнозначные по числу нодов, а разделются они по принципу поколения: желтые ноды - ноды моих сверстников, мятные - людей из старших параллелей и учителей. Музыкальных сообщества тоже два - голубое слева и зеленое справа - и тут уже разделение по тому, кто в каком коллективе играл (хотя между некоторыми людьми из этих сообществ есть общие связи). Кластер университетский - розовый внизу - един.
Вообще хочется сказать, что вот так смотришь на этот граф - и вся жизнь перед глазами. 

## Communities comparing

In [ ]:
nodes_in_community_one = [node for node,community in partition.items() if community == 1]
nodes_in_community_five = [node for node,community in partition.items() if community == 5]
graph_one = graph.subgraph(nodes_in_community_one)
graph_five = graph.subgraph(nodes_in_community_five)

In [ ]:
visualize_graph(graph_one, 'First Community', 'yellow')
metrics(graph_one)

In [ ]:
visualize_graph(graph_five, 'Fifth Community', 'lightpink')
metrics(graph_five)

Анализируя конкретно кластер моих сверстников и университетских однокурсников, я могу подтвердить свою ранее высказанную гипотезу о том, что в школе сообщество более плотное и тесное. Причем замечу, что число нодов в школе ниже, чем в университете. Также важно отметить, что значение диаметра - не лучший показатель для анализа графа, потому что мы видим, что в университете он ниже, хотя на анализе центральностей мы видели, что все важные ноды находятся в сообществе школы.

## Conclusion

Итак, еще до анализа своего графа социальных сетей у меня была гипотеза о том, что в школе граф будет плотнее и люди будут сильнее соединены. В ходе анализа графа, эта гипотеза подтвержилась. В целом все люди, у который высокие параметры центральности какой бы то ни было относятся к школе. 
Вторая гипотеза была, что в университете все общаются маленькими кучками, в то время как в школе существует одна большая тусовка. Это подтвердилось тем, что Eigenvector Centrality в школьном сообществе оказался выше.
Из важных наблюдений стоит отметить, что louvain method отлично разбивает граф на сообщества и что мой граф социальных сетей обладает таким же степенным распределением, что и графы социальных сетей, построенные другими людьми (приятно достигать результатов великих!). Также на моем графе подстверждается правило шести (или уже даже пяти) рукопожатий.
Наконец, хочу приложить свои визуализации из Gephi - одна с выделенными по модулярности сообществами (по сути то же, что и визуализация в louvain methods, только с подписями), вторая - с узлами, окрашенными согласно гендеру (мужчины в моем графе составляют треть, и большая их часть относится к школе и к сообществу музыкантов).

In [ ]:
friends_modularity = Image.open("Graph_of_friends.png")
display(friends_modularity)

In [ ]:
friends_modularity = Image.open("Friends_gender.png")
display(friends_modularity)